In [34]:
import pandas as pd
import numpy as np
import re

In [35]:
df = pd.read_csv('E:/NewDrive/DataScienceProjects/DataSets/Flipkart_dataset/Mi_new.csv')

In [36]:
df.duplicated().sum()

40

In [37]:
df_M= df.drop_duplicates()

In [38]:
df_M.shape

(350, 5)

In [39]:
pd.options.mode.chained_assignment = None  # default='warn'

In [40]:
df_M['Name']=df_M['Company Name'].str.extract('(Xiaomi|REDMI|Mi|Redmi)')
df_M['Name']=df_M['Name'].replace('REDMI','Redmi')
df_M['Name'].replace('', np.nan, inplace=True)
df_M.dropna(subset=['Name'], inplace=True)

In [41]:
df_M['test'] = df_M['Company Name'].str.extract('(\([^)]*\,)')
df_M['test1']=df_M['test'].astype(str)
df_M['Color'] = df_M['test1'].apply(lambda x: x.replace('(','').replace(',',''))

In [42]:
df_M['test2'] = df_M['Company Name'].str.split('(').str[0]
df_M['Model']= df_M['test2'].str.split().str[1:].str.join(' ')

In [43]:
def match(x):
    if re.search("Ratings",x):
        return x
    else:
        return "0"

In [44]:
df_M['test3']=df_M['Ratings'].apply(match)
df_M['test5']=df_M['test3'].apply(lambda x: x[x.find("SAMSUNG") : x.find("Ratings")])
df_M['Ratings(*)']=df_M['test3'].str.extract('(4\.\d|3\.\d)')
df_M['Ratings(*)']=df_M['Ratings(*)'].fillna(0)

In [45]:
df_M['test4']=df_M['test3'].apply(lambda x: x[x.find("Ratings") : x.find("Reviews")])
df_M['Reviews']=df_M['test4'].apply(lambda x: x.replace('Ratings','').replace('&','').replace(',',''))
df_M['Reviews']=df_M['Reviews'].str.strip()

In [46]:
def remove_whitespace(s):
    return ' '.join(s.split())

In [47]:
df_M['MobilesSold']=df_M['Reviews'].apply(remove_whitespace)
df_M['MobilesSold']=df_M['MobilesSold'].replace('','0')
df_M['MobilesSold']=df_M['MobilesSold'].astype(int)

In [48]:
df_M['test6']=df_M['Mobile Price'].apply(lambda x: x.replace('%','').replace('offFree','').replace('delivery','').replace('off','').replace('Free delivery','').replace('Free',''))

In [49]:
df_M['OfferInPerentage'] =df_M['offer'].str.extract('(\d+)')
df_M['OfferInPerentage']=df_M['OfferInPerentage'].fillna(0)
df_M['OfferInPerentage']=df_M['OfferInPerentage'].astype(int)

In [50]:
df_M['test7']=df_M['test6'].str.extract('(\₹[^)]*\₹)')

In [51]:
df_M['test7'] = np.where(df_M['test7'].isnull(),df_M['test6'],df_M['test7'])

In [52]:
df_M['DiscountedPrice']=df_M['test7'].apply(lambda x:str(x).replace('₹','').replace(',','').replace('Price: Not Available','nan'))
#Replacing NAN Values from Price(with discount) column
df_M['DiscountedPrice']=df_M['DiscountedPrice'].replace('nan','0')
df_M['DiscountedPrice']=df_M['DiscountedPrice'].astype(int)

In [53]:
df_M['MaxRetailPrice'] = (df_M.DiscountedPrice + ((df_M.OfferInPerentage/100) * df_M.DiscountedPrice))
df_M['MaxRetailPrice']=df_M['MaxRetailPrice'].round(0)

In [54]:
df_M['Sales(₹ CR)']=(df_M.DiscountedPrice*df_M.MobilesSold )/10000000
df_M['Sales(₹ CR)']=df_M['Sales(₹ CR)'].round(3)

In [55]:
df_M['test8']= df_M['Specifications'].str.split('|'and'ROM').str[0]
df_M[['RAM(GB)','ROM(GB)']]= df_M['test8'].str.split('|',expand = True)
df_M['RAM(GB)']=df_M['RAM(GB)'].apply(lambda x:str(x).replace('GB','').replace('RAM',''))
df_M['ROM(GB)']=df_M['ROM(GB)'].str.replace('GB','')

In [56]:
df_M['Display']= df_M['Specifications'].str.extract('\((.*?)\)')
df_M['Display']=df_M['Display'].str.replace('inch','')

In [57]:
df_M['Screen']=df_M['Specifications'].str.extract('(Quad HD+|HD+|Full HD+ AMOLED|QGVA|AMOLED)')
df_M['Screen']=df_M['Screen'].fillna("Not Mentioned")

In [58]:
df_M['test9']= df_M['Specifications'].apply(lambda x: x[x.find("Display"):x.find("Front")])

In [59]:
df_M['test10'] = df_M['test9'].apply(lambda x:str(x).replace('Display','').replace('+',''))

In [60]:
df_M[['Rear Camera','Front Camera']]= df_M['test10'].str.split('|',expand = True)

In [61]:
df_M['No of Rear Camera']= df_M['Rear Camera'].str.count("MP")
df_M['No of Front Camera']= df_M['Front Camera'].str.count("MP")
df_M['No of Front Camera']= df_M['No of Front Camera'].fillna(0)

In [62]:
df_M['Battery']= df_M['Specifications'].str.extract('(\d{4} mAh)')
df_M['Battery']= df_M['Battery'].apply(lambda x: str(x).replace('mAh',''))

In [63]:
df_M['Processor']= df_M['Specifications'].str.extract('(Exynos|Mediatek|Snapdragon|Octa Core|208MHz|8 Gen 1|UMS9230|quad core)')
df_M['Processor']=df_M['Processor'].fillna("Not Mentioned")

In [64]:
# drop all test columns and unwanted columns
df_M.drop(columns=['test','test1','test2','test3','test4','test5','test6','test7','test8','test9','test10','Company Name','Ratings','Specifications','Mobile Price','offer'],inplace=True,axis=1)

In [65]:
df_M.to_csv('E:/NewDrive/DataScienceProjects/DataSets/Flipkart_dataset/MiCleaned.csv',index=False)

In [66]:
#df_M.head(10)